In [ ]:
!pip install datasets
!pip install transformers
!pip install pyctcdecode
!pip install sentencepiece
!pip install librosa
!pip install tokenizers

!pip install https://github.com/kpu/kenlm/archive/master.zip

In [2]:
import os
import warnings

import librosa
import nltk
import numpy as np

import torch
from datasets import load_dataset
from transformers import Wav2Vec2ForCTC, Wav2Vec2ProcessorWithLM

nltk.download('punkt')

Model credits: https://huggingface.co/bond005/wav2vec2-large-ru-golos-with-lm

Модель 'wav2vec2-large-ru-golos-with-lm' хорошо распознаёт речь, но инференс не быстрый (30 сек инференс на ~30 сек аудио)

При этом модель '' распознаёт текст так же хорошо (на испольхованных примерах), но инференс проходит в два раза быстрее

In [31]:
'''
Audio file to array

path: path to audio file
sr: sampling rate (constant 16000 for current model)
'''

def speech_file_to_array_fn(path):
    speech_array, _ = librosa.load(path, sr=16000)
    return np.asarray(speech_array, dtype=np.float32)

In [41]:
'''
Speech to text inference

path: path to audio file
model: STT model object
processor: Wav2Vec2ProcessorWithLM object from transformers
num_processes: available number of CPU processes
'''

def speech_2_text(path, model, processor, num_processes=1):
    input_speech_array = speech_file_to_array_fn(path)

    inputs = processor(input_speech_array, sampling_rate=16_000,
                      return_tensors="pt", padding=True)

    with torch.no_grad():
        logits = model(inputs.input_values,
                       attention_mask=inputs.attention_mask).logits

    predicted_text = processor.batch_decode(logits=logits.numpy(),
                                            num_processes=num_processes
                                           ).text
    return predicted_text

In [ ]:
MODEL_ID = "bond005/wav2vec2-base-ru" # bond005/wav2vec2-large-ru-golos-with-lm
processor = Wav2Vec2ProcessorWithLM.from_pretrained(MODEL_ID)
model = Wav2Vec2ForCTC.from_pretrained(MODEL_ID)

In [ ]:
num_processes = max(1, os.cpu_count())

path = 'sample_2.wav'

In [43]:
predicted_text = speech_2_text(path=path, model=model,
                               processor=processor, num_processes=num_processes)

In [44]:
predicted_text

['переподготовка рабочих кадров бригадир дяжужный по переезду машинист машинист железнодорожно строительные машины машинист тепловоза из числа лиц имеющих высшее профессиональное или средние профессиональное образование машинист теплвоза работник по управление машинист электровоза мантер пути обходчик пути ремонтник сигналист стропальщик']

In [34]:
'''
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    for predicted_sentence in predicted_text):
        print("-" * 100)
        print("Prediction:", predicted_sentence)
'''

----------------------------------------------------------------------------------------------------
Prediction: (0, 'помощник машиниста электровоза чему вы научитесь проводить техническое обслуживание приемку и сдачу локомотива выполнять вспомогательные работы по управлению локомотивам и ведению поезда понимать причины возникновения неисправностей и уметь их исправлять')
